In [1]:
import os
from joblib import Parallel, delayed
import time
import numpy as np
import pickle
import pylab as pl
import pandas as pd

# import jupyternotify
import pylab as pl
import subprocess
import concurrent.futures
import shutil

# ip = get_ipython()
# ip.register_magics(jupyternotify.JupyterNotifyMagics)


In [22]:
        os.system('git add .; git commit -m "testing";git push --force origin HEAD:main')

    

[detached HEAD f3fdaf912] testing
 1 file changed, 1 deletion(-)


remote: This repository moved. Please use the new location:        
remote:   git@github.com:draznm/BftSmart_Final.git        
To github.com:tmane002/BftSmart_Final.git
 + 4e1d50130...f3fdaf912 HEAD -> main (forced update)


0

In [ ]:
# time.sleep(2000)

In [4]:

os.system('gcloud compute instances delete --zone=europe-west3-c --quiet $(gcloud compute instances list --format="value(name)")')
os.system('gcloud compute instances delete --zone=asia-south1-c --quiet $(gcloud compute instances list --format="value(name)")')
os.system('gcloud compute instances delete --zone=us-west1-b --quiet $(gcloud compute instances list --format="value(name)")')


ERROR: (gcloud.compute.instances.delete) argument INSTANCE_NAMES [INSTANCE_NAMES ...]: Must be specified.
Usage: gcloud compute instances delete INSTANCE_NAMES [INSTANCE_NAMES ...] [optional flags]
  optional flags may be  --delete-disks | --help | --keep-disks | --zone

For detailed information on this command and its flags, run:
  gcloud compute instances delete --help
ERROR: (gcloud.compute.instances.delete) argument INSTANCE_NAMES [INSTANCE_NAMES ...]: Must be specified.
Usage: gcloud compute instances delete INSTANCE_NAMES [INSTANCE_NAMES ...] [optional flags]
  optional flags may be  --delete-disks | --help | --keep-disks | --zone

For detailed information on this command and its flags, run:
  gcloud compute instances delete --help
ERROR: (gcloud.compute.instances.delete) argument INSTANCE_NAMES [INSTANCE_NAMES ...]: Must be specified.
Usage: gcloud compute instances delete INSTANCE_NAMES [INSTANCE_NAMES ...] [optional flags]
  optional flags may be  --delete-disks | --help | --k

512

In [5]:
config_list = [ 'join']



# config_list = ['cre_setup_2_scale_1_node_info']

# config_list2 = ['global_12cls',\
#                 'local_12cls',\
#                'global_3cls',\
#                'global_4cls',\
#                'global_6cls',\
#                'global_8cls',\
#                'global_2cls',\
#                'local_3cls',\
#                'local_4cls',\
#                'local_6cls',\
#                'local_8cls',\
#                'local_2cls']
# config_list2 = ['local_2cls']


config_list = config_list

In [6]:
config_list

['join']

In [7]:
for cfig in config_list:    
    if 2 > 1:
        
        if os.path.exists('node_info.csv'):
            os.remove('node_info.csv')

        os.system('cp node_setup/'+cfig+'.csv node_info.csv')


        df =pd.read_csv('node_info.csv')

        node_info_dict = df.to_dict()

        node_info_dict

        total_count = 0

        for key in node_info_dict['ServerOrClientregion'].keys():
            if node_info_dict['ServerOrClientregion'][key] ==0:
                total_count = total_count + 1

        total_count

        replica_count = 0

        for key in node_info_dict['cluster_id'].keys():
            if node_info_dict['cluster_id'][key] ==0:
                replica_count = replica_count + 1


        
        cluster_count = 0
        
        for key in node_info_dict['cluster_id']:
            print(key, node_info_dict['cluster_id'][key])
            if node_info_dict['cluster_id'][key] > cluster_count:
                cluster_count = node_info_dict['cluster_id'][key]
                
        cluster_count = cluster_count + 1
        
        n_clusters = cluster_count

        nclients = 0

        for i in node_info_dict['ServerOrClientregion']:
        #     print(node_info_dict['ServerOrClientregion'][i])
            if node_info_dict['ServerOrClientregion'][i] == 1:
                nclients = nclients + 1
        nclients

        clusters_tracked = []
        cluster_regions_tracked = []

        for key in node_info_dict['region']:
            if int(node_info_dict['cluster_id'][key]) not in clusters_tracked and int(node_info_dict['cluster_id'][key]) >=0:
                clusters_tracked.append(int(node_info_dict['cluster_id'][key]))
                cluster_regions_tracked.append(node_info_dict['region'][key].strip().replace('"', '')[1:-1])


        cluster_regions_tracked     

        cluster_regions_tracked, clusters_tracked


        regions = cluster_regions_tracked



        n_regions = len(regions)

        for i,k in enumerate(regions):
            print(i,k)

        for item, idx in enumerate(node_info_dict['region'].keys()):
            print(item, idx, node_info_dict['region'][item])

        lines  = []


        for idx, key in enumerate( node_info_dict['region'].keys()):

            line = 'gcloud compute instances create resdb-'+f"{idx:03}"+' --project=ucr-ursa-major-lesani-lab --zone='+node_info_dict['region'][key].strip()[1:-1]+' --machine-type=e2-standard-2 --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=goog-ec-src=vm_add-gcloud --reservation-affinity=any'
            lines.append(line)

        with open('launch_cloud_instances.sh', 'w') as f:
            for item in lines:
                f.write(str(item) + '\n')


        len(lines), node_info_dict['region'][key].strip()[1:-1]


        def run_command(command):
            subprocess.call(command, shell=True)

        commands = lines

        # Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
        with concurrent.futures.ThreadPoolExecutor(max_workers=60) as executor:
            # Submit each command to the executor for parallel execution
            futures = [executor.submit(run_command, command) for command in commands]

            # Wait for all tasks to complete
            concurrent.futures.wait(futures)

#         time.sleep(80)
#         time.sleep(180)
        

        # os.system('sh launch_cloud_instances.sh')

        for folder in os.listdir('.'):
            if 'config' in folder and folder!='config' and 'zip' not in folder and '.' not in folder:
                shutil.rmtree(folder)


        instance_type = 'e2-small'


        redirect = '>'

        os.system('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)" '+\
                  redirect+' all_internal_ips')



        os.system('gcloud compute instances list --format="value(networkInterfaces[0].networkIP)" '+\
              redirect+' all_external_ips')

        with open('all_internal_ips') as f:
            ipList = f.read().splitlines()
        ipList = [x for x in ipList if x!='None']

        ipList, len(ipList)

        new_server_only_ips= []
        new_client_only_ips= []



        for i in range(total_count + nclients):
            if node_info_dict['ServerOrClientregion'][i] ==0:
                new_server_only_ips = new_server_only_ips + [ipList[i]]
            else:
                new_client_only_ips = new_client_only_ips + [ipList[i]]



        new_server_only_ips, len(new_server_only_ips)

        new_client_only_ips, len(new_client_only_ips) 

        nclients = 0
        nservers = 0
        count = 0

        clusterIpMap = {}


        for i in node_info_dict['ServerOrClientregion']:
        #     print(node_info_dict['ServerOrClientregion'][i])

            if node_info_dict['ServerOrClientregion'][i] == 0:
                clusterIpMap[nservers] = ipList[count]
                nservers = nservers + 1

            count = count + 1

        nclients = 0
        nservers = 0
        count = 0

        for i in node_info_dict['ServerOrClientregion']:
        #     print(node_info_dict['ServerOrClientregion'][i])

            if node_info_dict['ServerOrClientregion'][i] == 1:
                print('test: ', 7001+nclients, ipList[count])
                clusterIpMap[7001+nclients] = ipList[count]
                nclients = nclients + 1

            count = count + 1

        clusterIpMap

        clusterPortMap1 = {}
        clusterPortMap2 = {}

        for i in range(len(ipList) - nclients):

            clusterPortMap1[i] = str(10000)

        for i in range(len(ipList) - nclients):
            clusterPortMap2[i] = str(20000)




        temp = 11900

        for i in range(nclients):

            clusterPortMap1[7001+i] = str(10000)
            clusterPortMap2[7001+i] = str(20000)

        clusterPortMap1, clusterPortMap2

        os.system('echo 3108 | sudo -S ./gradlew installDist')

        for i in range(total_count):
            os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(i)+' 1024')



        os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7002)+' 1024')
        os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7001)+' 1024')

        os.system('echo 3108 | sudo -S ./gradlew clean')

        for key in clusterPortMap1.keys():
            if key>7002:
        #         print(key)
                print()

                for folder in os.listdir('config/'):
                    if os.path.isdir('config/'+folder) and 'keys' in folder:
                        if os.path.exists('config/'+folder+'/'+'privatekey7001'):
                            shutil.copy('config/'+folder+'/'+'privatekey7001','config/'+folder+'/'+'privatekey'+str(key))
                            shutil.copy('config/'+folder+'/'+'publickey7001','config/'+folder+'/'+'publickey'+str(key))

                print('----------')
                print()

        for i in range(n_clusters):

            if os.path.exists('config'+str(i)):
                shutil.rmtree('config'+str(i))

            os.mkdir('config'+str(i))
            os.mkdir('config'+str(i)+'/keysRSA')
            os.mkdir('config'+str(i)+'/keysSunEC')
            os.mkdir('config'+str(i)+'/keysECDSA')



            shutil.copytree('config/workloads/', 'config'+str(i)+'/workloads')
            shutil.copytree('config/keysSSL_TLS/', 'config'+str(i)+'/keysSSL_TLS')


            for file in os.listdir('config/keys/'):

                shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysRSA/'+file)
                shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysSunEC/'+file)
                shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysECDSA/'+file)


            shutil.copy('config/system.config', 'config'+str(i)+'/system.config')    
            shutil.copy('config/logback.xml', 'config'+str(i)+'/logback.xml')





        clusterIpMap, clusterPortMap1

        def find_key_by_value(dictionary, value):
            for key, val in dictionary.items():
                if val == value:
                    return key
            # If the value is not found, you might want to handle this case accordingly.
            return None


        node_info_dict

        find_key_by_value(node_info_dict['ServerID'], 4)

        for n_cluster in range(n_clusters):

            with open('config'+str(n_cluster)+'/hosts.config','w') as file:

                for key in clusterIpMap:
                    if key < 7000:
                        file.writelines(str(key)+' '+clusterIpMap[key]+' '  +clusterPortMap1[key]+' ' +  clusterPortMap2[key]+' '+str(node_info_dict['cluster_id'][find_key_by_value(node_info_dict['ServerID'], key)] )+'\n')
                        print('test: ',str(key)+' '+clusterIpMap[key]+' '  +clusterPortMap1[key]+' ' +  clusterPortMap2[key]+' '+str(node_info_dict['cluster_id'][find_key_by_value(node_info_dict['ServerID'], key)] )+'\n')
                for key in clusterIpMap:
                    if key >= 7000:
                        file.writelines(str(key)+' '+clusterIpMap[key]+' '+clusterPortMap1[key]+'\n')
                        print('test: ', str(key)+' '+clusterIpMap[key]+' '+clusterPortMap1[key]+'\n')

                file.close()



        replica_count



        for n_cluster in range(n_clusters):
            with open('config'+str(n_cluster)+'/system.config','r') as file:
                data = file.readlines()


                nservers = 0
                server_id_list = []



                for i in node_info_dict['ServerOrClientregion']:
                    if node_info_dict['ServerOrClientregion'][i] == 0 and \
                    node_info_dict['cluster_id'][i]  == n_cluster and \
                    node_info_dict['join_col'][i] ==0:
                        nservers = nservers + 1
                        server_id_list.append(node_info_dict['ServerID'][i])

                f = int(np.ceil((nservers-1)/3))
                print('f is',f )


                for n_line in range(len(data)):
                    if ('system.servers.num' in data[n_line]) and '#' not in data[n_line]:


                        data[n_line] = 'system.servers.num = '+str(nservers)
                        print(data[n_line])

                    if ('system.servers.f' in data[n_line]) and '#' not in data[n_line]:
                        data[n_line] = 'system.servers.f = '+str(f)
                        print(data[n_line])


                    if ('system.initial.view' in data[n_line]) and '#' not in data[n_line]:
                        data[n_line] = 'system.initial.view = '
                        for iter_view in server_id_list:
                            data[n_line] = data[n_line] +str(iter_view)+','
        #                     data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
                            print(data[n_line])

                        data[n_line] = data[n_line][:-1]
                        print(data[n_line])



                file.close()

            with open('config'+str(n_cluster)+'/system.config', 'w') as f:
                for line in data:
                    f.write(f"{line}\n")


        for n_cluster in range(n_clusters):
            for i in range(999):

                for ncls in  range(1):



                    shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                                'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
                    shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
                                'config'+str(n_cluster)+'/keysRSA/'+'privatekey'+str(1000+(ncls*100)+i))

        os.system('git add .; git commit -m "Working Post Submission: testing";git push --force origin HEAD:main')
        

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 1
8 1
9 1
10 1
11 1
12 1
13 1
14 1
15 1
16 1
17 0
18 0
19 0
20 -1
21 -1
22 -1
23 -1
24 -1
0 us-west1-b
1 us-west1-b
0 0  'us-west1-b'
1 1  'us-west1-b'
2 2  'us-west1-b'
3 3  'us-west1-b'
4 4  'us-west1-b'
5 5  'us-west1-b'
6 6  'us-west1-b'
7 7  'us-west1-b'
8 8  'us-west1-b'
9 9  'us-west1-b'
10 10  'us-west1-b'
11 11  'us-west1-b'
12 12  'us-west1-b'
13 13  'us-west1-b'
14 14  'us-west1-b'
15 15  'us-west1-b'
16 16  'us-west1-b'
17 17  'us-west1-b'
18 18  'us-west1-b'
19 19  'us-west1-b'
20 20  'us-west1-b'
21 21  'us-west1-b'
22 22  'us-west1-b'
23 23  'us-west1-b'
24 24  'us-west1-b'


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-019].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-005].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-010].
 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20240614'.



NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
resdb-019  us-west1-b  e2-standard-2               10.138.0.2   34.83.201.243  RUNNING


 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20240614'.

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-003].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-022].
 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20240614'.



NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
resdb-010  us-west1-b  e2-standard-2               10.138.0.6   34.168.103.201  RUNNING
NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
resdb-005  us-west1-b  e2-standard-2               10.138.0.4   34.168.170.24  RUNNING


...

Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.



> Task :processResources NO-SOURCE
> Task :classes
> Task :jar
> Task :installDist

BUILD SUCCESSFUL in 9s
3 actionable tasks: 3 executed


[sudo] password for tejas: 

> Task :clean

BUILD SUCCESSFUL in 658ms
1 actionable task: 1 executed

----------


----------


----------

test:  0 10.138.0.7 10000 20000 0

test:  1 10.138.0.11 10000 20000 0

test:  2 10.138.0.18 10000 20000 0

test:  3 10.138.0.15 10000 20000 0

test:  4 10.138.0.32 10000 20000 0

test:  5 10.138.0.4 10000 20000 0

test:  6 10.138.0.30 10000 20000 0

test:  7 10.138.0.17 10000 20000 1

test:  8 10.138.0.16 10000 20000 1

test:  9 10.138.0.29 10000 20000 1

test:  10 10.138.0.6 10000 20000 1

test:  11 10.138.0.14 10000 20000 1

test:  12 10.138.0.10 10000 20000 1

test:  13 10.138.0.20 10000 20000 1

test:  14 10.138.0.9 10000 20000 1

test:  15 10.138.0.21 10000 20000 1

test:  16 10.138.0.5 10000 20000 1

test:  17 10.138.0.24 10000 20000 0

test:  18 10.138.0.19 10000 20000 0

test:  19 10.138.0.2 10000 20000 0

test:  7001 10.138.0.23 10000

test:  7002 10.138.0.22 10000

test:  7003 10.138.0.3 10000

test:  7004 10.138.0.12 10000

test:  7005 10.138.0.31 10000

test:  0 10.

remote: This repository moved. Please use the new location:        
remote:   git@github.com:draznm/BftSmart_Final.git        
To github.com:tmane002/BftSmart_Final.git
   7200c3f24..0b1aa9904  HEAD -> main


In [8]:
ipList

['10.138.0.7',
 '10.138.0.11',
 '10.138.0.18',
 '10.138.0.15',
 '10.138.0.32',
 '10.138.0.4',
 '10.138.0.30',
 '10.138.0.17',
 '10.138.0.16',
 '10.138.0.29',
 '10.138.0.6',
 '10.138.0.14',
 '10.138.0.10',
 '10.138.0.20',
 '10.138.0.9',
 '10.138.0.21',
 '10.138.0.5',
 '10.138.0.24',
 '10.138.0.19',
 '10.138.0.2',
 '10.138.0.23',
 '10.138.0.22',
 '10.138.0.3',
 '10.138.0.12',
 '10.138.0.31']

In [9]:
if 2>1:
    if 2>1:
        
#         time.sleep(30)

        # regions = [ 'us-west1-b', 'us-west1-b', 'us-west1-b']
        # regions = [ 'us-west1-b']


        instance_type = 'e2-small'

        df =pd.read_csv('node_info.csv')

        node_info_dict = df.to_dict()

        node_info_dict

        f = open('all_external_ips', "r")
        data1 = [line.strip() for line in f.readlines()]

        data1 = [x for x in data1 if x!='None']

        f = open('all_internal_ips', "r")
        data2 = [line.strip() for line in f.readlines()]

        data2 = [x for x in data2 if x!='None']

        data2, len(data2)

        data = []

        skip_instances = 0

        for i in range(skip_instances, len(data1)):
            if data1[i]!='None':
                data.append(data1[i] + '    ' + data2[i])

        data, len(data)

        for index, item in enumerate(data1):
            if 'publicIP' not in node_info_dict.keys():
                node_info_dict['publicIP'] = {}


            node_info_dict['publicIP'][index] = item 


        for index, item in enumerate(data2):
            if 'privateIP' not in node_info_dict.keys():
                node_info_dict['privateIP'] = {}

            node_info_dict['privateIP'][index] = item 



        for index, item in enumerate(data):
            if 'publicAndprivateIP' not in node_info_dict.keys():
                node_info_dict['publicAndprivateIP'] = {}

            node_info_dict['publicAndprivateIP'][index] = item 

        node_info_dict









        # time.sleep(60)

        total_count = 0

        for key in node_info_dict['ServerOrClientregion'].keys():
            if node_info_dict['ServerOrClientregion'][key] ==0:
                total_count = total_count + 1

        total_count





        def getTimeThps(file, flag):

            f = open(file, "r")
            data = f.readlines() 

            times = []
            thputs = []
            for line in data:
                if 'throughput[ops/s]' in line:
                    lineData = (line[13+line.find('max[ops/s])>'):-2])
                    print(lineData)

                    times.append(lineData.split('|')[0])
                    thputs.append(lineData.split('|')[-2])
            if (flag): 
                times.append(float(times[-1])+abs(float(times[-1]) - float(times[-2]) ))
                thputs.append(0)

            return np.array(times).astype(float), np.array(thputs).astype(float) 

        with open('all_external_ips') as f:
            lines = f.read().splitlines()


        lines = [x for x in lines if x!='None']
        lines, len(lines)

        for ip in lines:
            if ip!='None':
                current = (ip.split('.'))

                print('ssh -i "FinalKeys.pem" ubuntu@ec2-'+str(current[0])+'-'\
                      +str(current[1])+'-'+str(current[2])+'-'+str(current[3])+'.us-west-1.compute.amazonaws.com')

        ## Set number of clusters

        nclusters = 0

        for folder in os.listdir('.'):
            if 'config' in folder and folder!='config':
                nclusters = nclusters + 1


        print(nclusters)





        nreplicas = int(total_count/nclusters)
        
        nclients = 0
        
        for key in node_info_dict['ServerOrClientregion']:
            if node_info_dict['ServerOrClientregion'][key]==1:
                nclients = nclients + 1
        


        nodeIps = []
        clientIps = []

        for i in range(total_count + nclients):
            if node_info_dict['ServerOrClientregion'][i] ==0:
                nodeIps.append(lines[i])
            else:
                clientIps.append(lines[i])


        # nodeIps = lines[:nclusters* nreplicas]


        # no_cluster_yet_nodes = lines[nclusters* nreplicas:-(nclusters)]
        
        not_yet_joined = []
        
        for i in node_info_dict['node_id']:
        #     print(i)

            if node_info_dict['join_col'][i]==1:
                print(i)
                not_yet_joined.append(lines[i])


        experiment = 'bft_mc_'+cfig 

        # clientIps = lines[-(nclusters):]



        # clientIps = lines[-nclients:]



        clientIps, not_yet_joined
        
        

        node_info_dict

        node_info_dict['region'][0].strip()

        ## Functions

        def setup(i):

            ip = node_info_dict['publicIP'][i] 
            if ip!='None':
                current = (ip.split('.'))
                command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' commands.sh'+' resdb-'+f"{i:03}"+':/home/tejas/commands.sh'

                print(command)
                os.system(command)

        def initialize(i):
            ip = node_info_dict['publicIP'][i] 
            if ip!='None':
                current = (ip.split('.'))
                command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo sh commands.sh"'


                os.system(command)


        def git_clone(i):
            ip = lines[i]
            if ip!='None':
                current = (ip.split('.'))
                command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command " sudo git clone https://github.com/draznm/BftSmart_Final.git"'


                os.system(command)

        def git_pull(i):
            ip = node_info_dict['publicIP'][i] 
            if ip!='None':
                current = (ip.split('.'))
                command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "git config --global --add safe.directory /home/tejas/BftSmart_Final; cd BftSmart_Final; sudo git pull"'


                os.system(command)

        def compile_job(i):
            ip = node_info_dict['publicIP'][i] 
            if ip!='None':
                current = (ip.split('.'))
                command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd BftSmart_Final; sudo ./gradlew installDist"'


                os.system(command)



        def clean_job(i):
            ip = node_info_dict['publicIP'][i] 

            rm_command = 'sudo rm config/currentView; '
            for iter_ in range(nclusters):
                rm_command = rm_command + 'sudo rm config'+str(iter_)+'/currentView; '


            if ip!='None':
                current = (ip.split('.'))
                command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo rm nohup.out; cd BftSmart_Final; '+rm_command+'"'


                os.system(command)


        def clean_all(i):
            ip = node_info_dict['publicIP'][i] 

            rm_command = 'sudo rm -r BftSmart_Final; sudo rm nohup.out;sudo rm nohup_client.out;'



            if ip!='None':
                current = (ip.split('.'))
                command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo rm nohup.out; '+rm_command+'"'


                os.system(command)





        #     if ip!='None' and( ip in no_cluster_yet_nodes):

        #         current = (ip.split('.'))
        #         command = 'ssh  -o StrictHostKeyChecking=no -i /home/tejas/Documents/FKA.pem '+'ubuntu@ec2-'+str(current[0])+'-'\
        #               +str(current[1])+'-'+str(current[2])+'-'+str(current[3])+'.ap-south-1.compute.amazonaws.com -T "cd BftSmart_Final; sudo nohup ./runscripts/startReplicaYCSB.sh '+str(i)+' >../nohup.out 2>&1 &"'


        #         os.system(command)        
        # #         print(command)

        def check_node(i):
            ip = node_info_dict['publicIP'][i] 
            if ip!='None':
                current = (ip.split('.'))
                command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "tail -n 1000 nohup.out"'


                os.system(command)
        #         print(command)


        def run_basic_client(i):
            cmd_arg = ''

            ip = node_info_dict['publicIP'][i] 

            if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:

                current = (ip.split('.'))


                cmd_arg = str(node_info_dict['clientID'][i])



                current = (ip.split('.'))
                command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd BftSmart_Final; sudo nohup ./runscripts/ycsbClient.sh '+str(cmd_arg)+' >../nohup_client.out 2>&1 &"'


                os.system(command)


                
        def kill_java(i):
            ip = node_info_dict['publicIP'][i] 

            if ip!='None':
                current = (ip.split('.'))
                command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd BftSmart_Final; sudo killall java; sudo killall nohup; sudo killall hotstuff-app; sudo killall hotstuff-client;"'


                os.system(command)



        def kill_clients(i):
            ip = node_info_dict['publicIP'][i] 
            if ip!='None' and ip in clientIps:
                current = (ip.split('.'))
                command = 'ssh -o StrictHostKeyChecking=no -i /home/tejas/Documents/'+node_info_dict['key_file'][i].strip()[1:-1]+' '+'ubuntu@ec2-'+str(current[0])+'-'\
                      +str(current[1])+'-'+str(current[2])+'-'+str(current[3])+'.'+node_info_dict['region'][i].strip()[1:-1]+'.compute.amazonaws.com -T "cd BftSmart_Final; sudo killall java; sudo killall nohup"'


                os.system(command)

                


        ## First Time Setup

        lines, len(lines)

        # for i in range(10):
        #     setup(i)


        results = Parallel(n_jobs=60)(delayed(setup)(i) for i in range(len(lines)))
        print(results)  

        results = Parallel(n_jobs=60)(delayed(initialize)(i) for i in range(len(lines)))
        print(results)  

        # ## GIT Clone/Pull



        # # results = Parallel(n_jobs=60)(delayed(clean_all)(i) for i in range(len(lines)))
        # results = Parallel(n_jobs=60)(delayed(git_pull)(i) for i in range(len(lines)))
        # # results = Parallel(n_jobs=60)(delayed(git_clone)(i) for i in range(len(lines)))


        # print(results)  

        # ## Compile Job

        results = Parallel(n_jobs=60)(delayed(compile_job)(i) for i in range(len(lines)))
        print(results)  

        # ## CLEAN SERVERS

        # results = Parallel(n_jobs=60)(delayed(clean_job)(i) for i in range(len(lines)))
        # # results = Parallel(n_jobs=60)(delayed(clean_all)(i) for i in range(len(lines)))

        # print(results)  

        

                
    
    
    
    
    
    
    

ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-7.us-west-1.compute.amazonaws.com
ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-11.us-west-1.compute.amazonaws.com
ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-18.us-west-1.compute.amazonaws.com
ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-15.us-west-1.compute.amazonaws.com
ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-32.us-west-1.compute.amazonaws.com
ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-4.us-west-1.compute.amazonaws.com
ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-30.us-west-1.compute.amazonaws.com
ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-17.us-west-1.compute.amazonaws.com
ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-16.us-west-1.compute.amazonaws.com
ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-29.us-west-1.compute.amazonaws.com
ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-6.us-west-1.compute.amazonaws.com
ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-14.us-west-1.compute.amazonaws.com
ssh -i "FinalKeys.pem" ubuntu@ec2-10-138-0-10.us-west-1.compute.ama

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...
Cloning into 'BftSmart_Final'...


Hit:1 http://us-west1.gce.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://us-west1.gce.archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:3 http://us-west1.gce.archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:4 http://us-west1.gce.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:6 http://us-west1.gce.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Hit:1 http://us-west1.gce.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://us-west1.gce.archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:3 http://us-west1.gce.archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:7 http://us-west1.gce.archive.ubuntu.com/ubuntu focal/universe amd64 c-n-f Metadata [265 kB]
Get:8 http://us-west1.gce.archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [144 kB]
Get:9 http://us-west1.gce.archive.ubuntu.com/ubuntu focal/multiverse Tra

Get:32 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [456 kB]
Get:33 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [14.0 kB]
Get:34 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2956 kB]
Get:35 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [413 kB]
Get:36 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 c-n-f Metadata [544 B]
Get:37 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [991 kB]
Get:38 http://security.ubuntu.com/ubuntu focal-security/universe Translation-en [210 kB]
Get:39 http://security.ubuntu.com/ubuntu focal-security/universe amd64 c-n-f Metadata [20.9 kB]
Get:40 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [24.8 kB]
Get:41 http://security.ubuntu.com/ubuntu focal-security/multiverse Translation-en [5968 B]
Get:42 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 c-n-f Metadata

...


> Task :compileJava
> Task :jar

> Task :compileJava


Note: /home/tejas/BftSmart_Final/src/main/java/bftsmart/reconfiguration/ViewController.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.



> Task :compileJava
> Task :installDist

BUILD SUCCESSFUL in 32s
3 actionable tasks: 3 executed

> Task :processResources NO-SOURCE
> Task :classes

> Task :compileJava
> Task :jar


Note: /home/tejas/BftSmart_Final/src/main/java/bftsmart/reconfiguration/ViewController.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.



> Task :processResources NO-SOURCE
> Task :classes

> Task :processResources NO-SOURCE
> Task :classes
> Task :jar

> Task :processResources NO-SOURCE
> Task :classes

> Task :processResources NO-SOURCE
> Task :classes
> Task :jar

> Task :processResources NO-SOURCE
> Task :classes
> Task :jar
> Task :installDist

BUILD SUCCESSFUL in 33s
3 actionable tasks: 3 executed
> Task :jar

> Task :processResources NO-SOURCE
> Task :classes
> Task :jar
> Task :installDist

BUILD SUCCESSFUL in 34s
3 actionable tasks: 3 executed

> Task :processResources NO-SOURCE
> Task :classes
> Task :jar
> Task :installDist

> Task :processResources NO-SOURCE
> Task :classes

> Task :processResources NO-SOURCE
> Task :classes

BUILD SUCCESSFUL in 34s
3 actionable tasks: 3 executed

> Task :processResources NO-SOURCE
> Task :classes
> Task :jar

> Task :processResources NO-SOURCE
> Task :classes
> Task :installDist

BUILD SUCCESSFUL in 34s
3 actionable tasks: 3 executed
> Task :jar
> Task :jar
> Task :installD

In [10]:
# os.system('git add .; git commit -m "reverting back";git push --force origin HEAD:main')


In [11]:
not_yet_joined

['10.138.0.24', '10.138.0.19', '10.138.0.2']

In [12]:
        join_counter = 0
    
        experiment = 'bft_mc_'+cfig+'_default' 

        def join_nodes(i, joining_node_num, port1, port2, config_num):
            global join_counter
            join_counter = join_counter + 1
            cmd_arg = ''

            ip = node_info_dict['publicIP'][i] 
            print('hlea', node_info_dict['ServerOrClientregion'][i])

            if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
                print('dasfdik')

                current = (ip.split('.'))


                cmd_arg = str(node_info_dict['clientID'][i])

                if int(node_info_dict['clientID'][i])==1:


                    current = (ip.split('.'))
                    command = 'nohup gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd BftSmart_Final; sudo nohup ./runscripts/smartrun.sh bftsmart.reconfiguration.util.DefaultVMServices '+\
                    str(len(joining_node_num))+' '

                    for jnn in joining_node_num:

                        command = command + ' '+ str(17+jnn)+' '+\
                        str(not_yet_joined[jnn])+' '+str(port1)+' '+str(port2)+' config'+str(config_num)+ ' '


                    command = command + ' >../nohup_join_'+str(join_counter)+'.out 2>&1 &"'

                    os.system(command)        
    
        def leave_nodes(i, joining_node_num, port1, port2, config_num):
            global join_counter
            join_counter = join_counter + 1
            cmd_arg = ''

            ip = node_info_dict['publicIP'][i] 
            print('hlea', node_info_dict['ServerOrClientregion'][i])

            if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
                print('dasfdik')

                current = (ip.split('.'))


                cmd_arg = str(node_info_dict['clientID'][i])

                if int(node_info_dict['clientID'][i])==1:


                    current = (ip.split('.'))
                    command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd BftSmart_Final; sudo nohup ./runscripts/smartrun.sh bftsmart.reconfiguration.util.DefaultVMServices '+\
                    ' '

                    for jnn in joining_node_num:

                        command = command + ' '+ str(17+jnn)+' '


                    command = command + ' >../nohup_leave_'+str(join_counter)+'.out 2>&1 &"'

                    os.system(command)                        
    
        results = Parallel(n_jobs=60)(delayed(kill_java)(i) for i in range(len(lines)))

        def run_server_node(i):
            ip = node_info_dict['publicIP'][i] 
        #     if ip!='None' and( ip in nodeIps or ip in no_cluster_yet_nodes):
            if ip!='None' and node_info_dict['ServerOrClientregion'][i]==0:

                current = (ip.split('.'))
                command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd BftSmart_Final; sudo nohup ./runscripts/startReplicaYCSB.sh '+str(node_info_dict['ServerID'][i])+' >../nohup.out 2>&1 &"'


                os.system(command)


                
                
        def kill_clients(i):
            ip = node_info_dict['publicIP'][i] 

            if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
                current = (ip.split('.'))
                command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd BftSmart_Final; sudo killall java; sudo killall nohup; sudo killall hotstuff-app; sudo killall hotstuff-client;"'


                os.system(command)

        # for i in node_info_dict['publicIP']:
        #     run_server_node(i)

        ## RUN SERVERS

        # results = Parallel(n_jobs=60)(delayed(run_server_node)(i) for i in range(len(lines)))
        # print(results)  

        ### CHECK NODE RESULT

        # check_node(0)

        # ## Run basic client

        # # time.sleep(20)
        # results = Parallel(n_jobs=60)(delayed(run_basic_client)(i) for i in range(len(lines)))
        # print(results)  

        # check_node(0)

        # time.sleep(60)

        # results = Parallel(n_jobs=60)(delayed(kill_java)(i) for i in range(len(lines)))

        # results = Parallel(n_jobs=60)(delayed(clean_job)(i) for i in range(len(lines)))

        ## Main Job

        # os.system('git add .; git commit -m "test";git push')


        # time.sleep(60)

        os.system('git add .; git commit -m "testing reconfig perf";git push --force origin HEAD:main')

# if 2>1:
#     if 2>1:
        
        #     %%time


        # results = Parallel(n_jobs=60)(delayed(clean_all)(i) for i in range(len(lines)))

        results = Parallel(n_jobs=60)(delayed(kill_java)(i) for i in range(len(lines)))

        results = Parallel(n_jobs=60)(delayed(git_pull)(i) for i in range(len(lines)))
        # results = Parallel(n_jobs=60)(delayed(git_clone)(i) for i in range(len(lines)))

        results = Parallel(n_jobs=60)(delayed(compile_job)(i) for i in range(len(lines)))
        # print(results) 





        results = Parallel(n_jobs=60)(delayed(clean_job)(i) for i in range(len(lines)))
        # results = Parallel(n_jobs=60)(delayed(clean_all)(i) for i in range(len(lines)))


        results = Parallel(n_jobs=60)(delayed(run_server_node)(i) for i in range(len(lines)))
        # print(results)  


        print('XXXXXXXX Started Servers XXXXXXXXXXXX')

nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
nohup: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
hotstuff-app: no process found
nohup: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-app: no process found


[detached HEAD 4e1d50130] testing reconfig perf
 1 file changed, 42355 insertions(+), 5 deletions(-)


remote: This repository moved. Please use the new location:        
remote:   git@github.com:draznm/BftSmart_Final.git        
To github.com:tmane002/BftSmart_Final.git
   0b1aa9904..4e1d50130  HEAD -> main
java: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
java: no process found
nohup: no process found
java: no process found
hotstuff-app: no process found
java: no process found
nohup: no process found
hotstuff-client: no process found
hotstuff-app: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-client: no process found
java: no process found
java: no process found
nohup: no process found
hotstuff-app: no process found
nohup: no process found
hotstuff-app: no process found
java: no process found
hotstuff-client: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-client: no process found
java: no 

Updating 0b1aa990..4e1d5013
Fast-forward
Updating 0b1aa990..4e1d5013
Fast-forward
 default_joins.ipynb | 42360 +++++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 42355 insertions(+), 5 deletions(-)
 default_joins.ipynb | 42360 +++++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 42355 insertions(+), 5 deletions(-)
Updating 0b1aa990..4e1d5013
Fast-forward
Updating 0b1aa990..4e1d5013
Fast-forward
 default_joins.ipynb | 42360 +++++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 42355 insertions(+), 5 deletions(-)
 default_joins.ipynb | 42360 +++++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 42355 insertions(+), 5 deletions(-)


From https://github.com/tmane002/BftSmart_Final
   0b1aa990..4e1d5013  main       -> origin/main


...

From https://github.com/tmane002/BftSmart_Final
   0b1aa990..4e1d5013  main       -> origin/main
From https://github.com/tmane002/BftSmart_Final
   0b1aa990..4e1d5013  main       -> origin/main
From https://github.com/tmane002/BftSmart_Final
   0b1aa990..4e1d5013  main       -> origin/main
From https://github.com/tmane002/BftSmart_Final
   0b1aa990..4e1d5013  main       -> origin/main
From https://github.com/tmane002/BftSmart_Final
   0b1aa990..4e1d5013  main       -> origin/main
From https://github.com/tmane002/BftSmart_Final
   0b1aa990..4e1d5013  main       -> origin/main
From https://github.com/tmane002/BftSmart_Final
   0b1aa990..4e1d5013  main       -> origin/main
From https://github.com/tmane002/BftSmart_Final
   0b1aa990..4e1d5013  main       -> origin/main
From https://github.com/tmane002/BftSmart_Final
   0b1aa990..4e1d5013  main       -> origin/main
From https://github.com/tmane002/BftSmart_Final
   0b1aa990..4e1d5013  main       -> origin/main


Updating 0b1aa990..4e1d5013
Fast-forward
Updating 0b1aa990..4e1d5013
Fast-forward
Updating 0b1aa990..4e1d5013
Fast-forward
 default_joins.ipynb | 42360 +++++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 42355 insertions(+), 5 deletions(-)
 default_joins.ipynb | 42360 +++++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 42355 insertions(+), 5 deletions(-)
Updating 0b1aa990..4e1d5013
Fast-forward
 default_joins.ipynb | 42360 +++++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 42355 insertions(+), 5 deletions(-)
Updating 0b1aa990..4e1d5013
Fast-forward
 default_joins.ipynb | 42360 +++++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 42355 insertions(+), 5 deletions(-)
 default_joins.ipynb | 42360 +++++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 42355 insertions(+), 5 deletions(-)
Updating 0b1aa990..4e1d5013
Fast-forward
Updating 0b1aa990..4e1d5013
Fast-forward
Updating 0b1aa990..4e1d5013
Fast-forward
 defaul


BUILD SUCCESSFUL in 12s
3 actionable tasks: 3 up-to-date


rm: cannot remove 'nohup.out': No such file or directory
rm: cannot remove 'config/currentView': No such file or directory
rm: cannot remove 'config0/currentView': No such file or directory
rm: cannot remove 'config1/currentView': No such file or directory
rm: cannot remove 'nohup.out': No such file or directory
rm: cannot remove 'nohup.out': No such file or directory
rm: cannot remove 'config/currentView': No such file or directory
rm: cannot remove 'config0/currentView': No such file or directory
rm: cannot remove 'config/currentView': No such file or directory
rm: cannot remove 'config1/currentView': No such file or directory
rm: cannot remove 'config0/currentView': No such file or directory
rm: cannot remove 'config1/currentView': No such file or directory
rm: cannot remove 'nohup.out': No such file or directory
rm: cannot remove 'config/currentView': No such file or directory
rm: cannot remove 'config0/currentView': No such file or directory
rm: cannot remove 'config1/currentView'

XXXXXXXX Started Servers XXXXXXXXXXXX


In [13]:
if 2>1:
    if 2>1:
        
        time.sleep(30)
        

        check_node(7)

        print('XXXXXXXX Started Clients XXXXXXXXXXXX')

#         results = Parallel(n_jobs=60)(delayed(run_basic_client)(i) for i in range(len(lines)))
        results = Parallel(n_jobs=60)(delayed(run_basic_client)(i) for i in [20,21,24])

        
#         for i in [8]:
#             ip = node_info_dict['publicIP'][i] 


#             if ip!='None' and node_info_dict['ServerOrClientregion'][i]==0:
#                 current = (ip.split('.'))
#                 command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/BftSmart_Final/config0/currentView ../Experiments/'+experiment+'/currentView'
#                 print(command)
#                 os.system(command)
                
                
        def copyCurrentView(config_num):

            ip = node_info_dict['publicIP'][i] 
            if ip!='None':
                current = (ip.split('.'))
                command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' commands.sh'+' resdb-'+f"{i:03}"+':/home/tejas/commands.sh'

                print(command)
                os.system(command)

        


#         time.sleep(40)
        
#         time.sleep(30)
        time.sleep(250)
    
        results = Parallel(n_jobs=60)(delayed(kill_clients)(i) for i in range(len(lines)))

        results = Parallel(n_jobs=60)(delayed(kill_java)(i) for i in range(len(lines)))

        check_node(0)

        if not os.path.exists('../Experiments/'+experiment):
            os.mkdir('../Experiments/'+experiment)

        os.system('cd ../Experiments/'+experiment+';rm *;cd ../; rm '+experiment+'.png')



        for i in node_info_dict['publicIP']:
            ip = node_info_dict['publicIP'][i] 


            if ip!='None' and node_info_dict['ServerOrClientregion'][i]==0:
                current = (ip.split('.'))
                command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup.out ../Experiments/'+experiment+'/nohup_c'+str(nreplicas)+'_'+str(node_info_dict['ServerID'][i])+'.out'

                os.system(command)

                break;
                
                

        for i in node_info_dict['publicIP']:
            ip = node_info_dict['publicIP'][i] 



            if ip!='None' and node_info_dict['ServerOrClientregion'][i]==0:
                print('client copying id= ',i)
                current = (ip.split('.'))
                command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/BftSmart_Final/data.json ../Experiments/'+experiment+'/nohup_times_'+str(i)+'.out'
                print(command)
                os.system(command)
                
                break




        for i in node_info_dict['publicIP']:
            ip = node_info_dict['publicIP'][i] 



            if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
                print('client copying id= ',i)
                current = (ip.split('.'))
                command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup_client.out ../Experiments/'+experiment+'/nohup_client_'+str(node_info_dict['clientID'][i])+'.out'
                print(command)
                os.system(command)


        for i in node_info_dict['publicIP']:
            ip = node_info_dict['publicIP'][i] 


            print(i)
            if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
                print('client copying id= ',i)


        node_info_dict

        clientIps




        def getTimesThputs(file):
            f = open(file, "r")
            data = f.readlines() 
            times = []
            thputs = []
            latencies = []
            for line in data:
        #         print(line)
                if not line.startswith('--') and 'current ops/sec; ' in line and 'AverageLatency(us)=' in line and not 'JVM' in line:

        #         print(line)
                    times.append(line.split('sec:')[0].strip())
                    thputs.append(line.split(';')[1].split('current')[0].strip())
                
                
                    read_lat = '0'
                    write_lat = '0'
                    
                    if ('READ AverageLatency' in line):
                        
                        print(line)

                        read_lat = line.split('READ AverageLatency(us)=')[1].split(']')[0].strip()
                        
                        
                    if ('UPDATE AverageLatency' in line):                    
                        
                        write_lat = line.split('UPDATE AverageLatency(us)=')[1].split(']')[0].strip()


                    if (('READ AverageLatency' not in line) and ('UPDATE AverageLatency' not in line)):
                        latencies.append(line.split('=')[1].split(']')[0].strip())
                    else:
                        read_frac = 0
                        write_frac = 0
                        
                        if float(read_lat) > 0:
                            read_frac = 0.85
                            
                        if float(write_lat) > 0:
                            write_frac = 0.15
                            
#                         print(line, write_frac, read_frac, float(read_lat), float(write_lat))
                        
                        lat = str( (write_frac * float(write_lat) +\
                                    read_frac*float(read_lat))/(write_frac+read_frac)\
                                 )
                        latencies.append(lat)

                    print(line,latencies[-1],thputs[-1],times[-1])

            return np.array(times).astype(float), np.array(thputs).astype(float), np.array(latencies).astype(float)/1e6  


        times_thputsDict = {}
        times_latsDict = {}


#         for ip in clientIps[:1]:    
#             i = clientIps.index(ip)

#             times ,thputs, lats = getTimesThputs('../Experiments/'+experiment+'/nohup_client_'+str(i)+'.out')

#             print(times, thputs)
#             for i in range(len(times)):
#                 times_thputsDict[times[i]] = thputs[i]
#                 times_latsDict[times[i]] = lats[i]


#         for ip in clientIps[1:]:    
#             i = clientIps.index(ip)

#             times ,thputs, lats = getTimesThputs('../Experiments/'+experiment+'/nohup_client_'+str(i)+'.out')


#             for i in range(len(times)):

#                 if times[i] in times_thputsDict.keys():
#                     times_thputsDict[times[i]] += thputs[i]

#                 if times[i] in times_latsDict.keys():
#                     times_latsDict[times[i]] += lats[i]


#         times_thputsDict



#         PIK = '../Experiments/'+experiment+"/pickle.dat"

#         data = [times_thputsDict, times_latsDict]

#         with open(PIK, "wb") as f:
#             pickle.dump(data, f)

#         experiment

#         # experiment = 'TestRun2_8_clusters_2'
#         # PIK = '../Experiments/'+experiment+"/pickle.dat"

#         data2 = []
        
#         with open(PIK, "rb") as f:
#             data2 =  pickle.load(f)


#         dict1 = data2[0]
#         dict2 = data2[1]



#         times1 = np.array(list(dict1.keys()))
#         thputs = np.array(list(dict1.values()))

#         times2 = np.array(list(dict2.keys()))
#         lats = np.array(list(dict2.values()))

        

        

#         times_new = [x for x in range(1+int(times1[-1]))]
#         thputs_new = [0 for x in range(1+int(times1[-1]))]

#         for idx, t in enumerate(times1):
#             thputs_new[int(t)] = thputs[idx]

    

#         pl.figure(figsize = (12,8))
#         pl.rcParams.update({'font.size': 17})
#         pl.plot(times_new[:], thputs_new[:], '-o', label='BfTSmart-MC')


#         pl.title(r'Throughput')


#         pl.xlabel('Time (s)')
#         pl.ylabel('Throughput (Txn/s)')
#         pl.legend()
#         pl.savefig('../Experiments/'+experiment+'/out1.png', dpi = 150)
#         pl.show()




#         pl.figure(figsize = (12,8))
#         pl.rcParams.update({'font.size': 17})
#         pl.plot(times1[:], lats[:]/ len(clientIps), '-o', label='BfTSmart-MC')


#         pl.title(r'Latency')


#         pl.xlabel('Time (s)')
#         pl.ylabel('Latency (Txn/s)')
#         pl.legend()
#         pl.savefig('../Experiments/'+experiment+'/out2.png', dpi = 150)
#         pl.show()

#         times1, lats

#         lats, np.average(lats[-85:-5])/len(clientIps)

#         np.sum(thputs),  np.average(thputs[-85:-5])

#         thputs

#         np.average(thputs[60:80]), np.average(lats[60:80])/nclusters, np.average(thputs[-35:-5]), np.average(lats[-35:-5])

#         # np.average(lats[-50:]), np.average(thputs[-50:]) # (0.0542783668, 166.87759999999997)

#         np.sum((thputs[:180]))

#         np.sum((thputs[:180]))
        
        
        for i in node_info_dict['publicIP']:
            ip = node_info_dict['publicIP'][i] 


            if ip!='None' and node_info_dict['ServerOrClientregion'][i]==0:
                current = (ip.split('.'))
                command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup.out ../Experiments/'+experiment+'/nohup_c'+str(nreplicas)+'_'+str(node_info_dict['ServerID'][i])+'.out'

                os.system(command)

                break;
                
                
                
        for i in node_info_dict['publicIP']:
            ip = node_info_dict['publicIP'][i] 


            if ip!='None' and node_info_dict['join_col'][i]==1:
                current = (ip.split('.'))
                command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup.out ../Experiments/'+experiment+'/nohup_c'+str(nreplicas)+'_'+str(node_info_dict['ServerID'][i])+'.out'

                os.system(command)


        for i in node_info_dict['publicIP']:
            ip = node_info_dict['publicIP'][i] 

            for iter_ in range(join_counter+1):
                
                if ip!='None' and int(node_info_dict['clientID'][i])==1:
                    current = (ip.split('.'))
                    command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup_join_'+str(iter_)+'.out ../Experiments/'+experiment+'/nohup_j_'+str(iter_)+'.out'
                    os.system(command)

                    command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup_leave_'+str(iter_)+'.out ../Experiments/'+experiment+'/nohup_l_'+str(iter_)+'.out'
                    os.system(command)



#         os.system('gcloud compute instances delete --zone=europe-west3-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=asia-south1-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=us-west1-b --quiet $(gcloud compute instances list --format="value(name)")')

#     except:
        
#         os.system('gcloud compute instances delete --zone=europe-west3-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=asia-south1-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=us-west1-b --quiet $(gcloud compute instances list --format="value(name)")')
        
#         continue
    

config+cinfo.getClusterNumber(id) is: config1
-- Creating current view from configuration file
-- myPort, InetAddress.getByName(myAddress) and myAddress are: 20000, /10.138.0.17, 10.138.0.17
-- getConnection HERE 
-- SSL/TLS handshake complete!, Id:0  ## CipherSuite: TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256.
-- Reached here inside Server Connection
-- getConnection HERE 
-- Reached here inside Server Connection
-- SSL/TLS handshake complete!, Id:1  ## CipherSuite: TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256.
-- getConnection HERE 
-- Reached here inside Server Connection
-- SSL/TLS handshake complete!, Id:2  ## CipherSuite: TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256.
-- getConnection HERE 
-- Reached here inside Server Connection
-- SSL/TLS handshake complete!, Id:3  ## CipherSuite: TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256.
-- getConnection HERE 
-- Reached here inside Server Connection
-- SSL/TLS handshake complete!, Id:4  ## CipherSuite: TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256.
-- getConnecti

gcloud compute scp --zone "us-west1-b" commands.sh resdb-008:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-020:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-014:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-021:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-010:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-002:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-004:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-000:/home/tejas/commands.sh


nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-client: no process found


gcloud compute scp --zone "us-west1-b" commands.sh resdb-009:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-016:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-012:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-018:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-011:/home/tejas/commands.sh


nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process found
nohup: no process found
hotstuff-app: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-client: no process found
nohup: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-client: no process found
nohup: no process found
hotstuff-app: no process found
hotstuff-client: no process found
nohup: no process fou

...

scp: /home/tejas/nohup_client.out: No such file or directory
ERROR: (gcloud.compute.scp) [/usr/bin/scp] exited with return code [1].


client copying id=  23
gcloud compute scp --zone "us-west1-b" resdb-023:/home/tejas/nohup_client.out ../Experiments/bft_mc_join_default/nohup_client_3.out


scp: /home/tejas/nohup_client.out: No such file or directory
ERROR: (gcloud.compute.scp) [/usr/bin/scp] exited with return code [1].


client copying id=  24
gcloud compute scp --zone "us-west1-b" resdb-024:/home/tejas/nohup_client.out ../Experiments/bft_mc_join_default/nohup_client_4.out
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
client copying id=  20
21
client copying id=  21
22
client copying id=  22
23
client copying id=  23
24
client copying id=  24


scp: /home/tejas/nohup_join_0.out: No such file or directory
ERROR: (gcloud.compute.scp) [/usr/bin/scp] exited with return code [1].
scp: /home/tejas/nohup_leave_0.out: No such file or directory
ERROR: (gcloud.compute.scp) [/usr/bin/scp] exited with return code [1].


In [14]:
#         for i in node_info_dict['publicIP']:
#             ip = node_info_dict['publicIP'][i] 



#             if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
#                 print('client copying id= ',i)
#                 current = (ip.split('.'))
#                 command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup_client.out ../Experiments/'+experiment+'/nohup_client_'+str(node_info_dict['clientID'][i])+'.out'
#                 print(command)
#                 os.system(command)


In [15]:
for i in [ 17,1,7]:
# for i in [ 8,9,10,11,12,13,14,15,16,18,19,2,3,4,5,6]:
    
    ip = node_info_dict['publicIP'][i] 


    if ip!='None' and node_info_dict['ServerOrClientregion'][i]==0:
        current = (ip.split('.'))
        command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup.out ../Experiments/'+experiment+'/nohup_c'+str(nreplicas)+'_'+str(node_info_dict['ServerID'][i])+'.out'
        print(command)
        os.system(command)

#                 break;
                

gcloud compute scp --zone "us-west1-b" resdb-017:/home/tejas/nohup.out ../Experiments/bft_mc_join_default/nohup_c10_17.out
gcloud compute scp --zone "us-west1-b" resdb-001:/home/tejas/nohup.out ../Experiments/bft_mc_join_default/nohup_c10_1.out
gcloud compute scp --zone "us-west1-b" resdb-007:/home/tejas/nohup.out ../Experiments/bft_mc_join_default/nohup_c10_7.out


In [16]:
#         for i in node_info_dict['publicIP']:
#             ip = node_info_dict['publicIP'][i] 


#             if ip!='None' and node_info_dict['ServerOrClientregion'][i]==0:
#                 current = (ip.split('.'))
#                 command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup.out ../Experiments/'+experiment+'/nohup_c'+str(nreplicas)+'_'+str(node_info_dict['ServerID'][i])+'.out'

#                 os.system(command)

# #                 break;
                
                

In [17]:
# for i in node_info_dict['publicIP']:
#     ip = node_info_dict['publicIP'][i] 



#     if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
#         print('client copying id= ',i)
#         current = (ip.split('.'))
#         command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup_client.out ../Experiments/'+experiment+'/nohup_client_'+str(node_info_dict['clientID'][i])+'.out'
#         print(command)
#         os.system(command)

In [18]:

# def join_node(i, joining_node_num, port1, port2, config_num):
#     cmd_arg = ''

#     ip = node_info_dict['publicIP'][i] 

#     if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:

#         current = (ip.split('.'))


#         cmd_arg = str(node_info_dict['clientID'][i])

#         if int(node_info_dict['clientID'][i])==1:


#             current = (ip.split('.'))
#             command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+\
#             '" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd BftSmart_Final; sudo nohup ../runscripts/sm`artrun.sh bftsmart.reconfiguration.util.DefaultVMServices 8 '+\
#             str(not_yet_joined[joining_node_num])+' '+str(port1)+' '+str(port2)+' config'+str(config_num)+' >../dump_files/nohup_join'+\
#             str(joining_node_num)+'.out 2>&1 &"'

#             print(command)

In [19]:
# import os
# os.system('gcloud compute instances delete --zone=europe-west3-c --quiet $(gcloud compute instances list --format="value(name)")')
# os.system('gcloud compute instances delete --zone=asia-south1-c --quiet $(gcloud compute instances list --format="value(name)")')
# os.system('gcloud compute instances delete --zone=us-west1-b --quiet $(gcloud compute instances list --format="value(name)")')


In [20]:
#         for i in node_info_dict['publicIP']:
#             ip = node_info_dict['publicIP'][i] 



#             if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
#                 print('client copying id= ',i)
#                 current = (ip.split('.'))
#                 command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup_client.out ../Experiments/'+experiment+'/nohup_client_'+str(node_info_dict['clientID'][i])+'.out'
#                 print(command)
#                 os.system(command)

In [21]:
#         for i in node_info_dict['publicIP']:
#             ip = node_info_dict['publicIP'][i] 



#             if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
#                 print('client copying id= ',i)
#                 current = (ip.split('.'))
#                 command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup_client.out ../Experiments/'+experiment+'/nohup_client_'+str(node_info_dict['clientID'][i])+'.out'
#                 print(command)
#                 os.system(command)

gcloud compute scp --zone "us-west1-b" commands.sh resdb-003:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-001:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-013:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-006:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-019:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-023:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-017:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-015:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-024:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-005:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-007:/home/tejas/commands.sh
gcloud compute scp --zone "us-west1-b" commands.sh resdb-022:/hom